In [ ]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# First import the modules
import aicons.bayesbrainGPT.bayes_brain
import aicons.bayesbrainGPT.utility_function.utility_base
import aicons.bayesbrainGPT.utility_function.marketing_utility

# Now reload them for development/testing
importlib.reload(aicons.bayesbrainGPT.bayes_brain)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)
importlib.reload(aicons.bayesbrainGPT.utility_function.marketing_utility)

# Import the classes after reloading
from aicons.bayesbrainGPT.bayes_brain import BaseAIcon
from aicons.bayesbrainGPT.utility_function.marketing_utility import MarketingROIUtility

# Create the AIcon
aicon = BaseAIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

# Add some continuous factors
aicon.add_continuous_factor("conversion_rate", 0.05, 0.02)  # mean=0.05, std=0.02
aicon.add_continuous_factor("cost_per_click", 1.0, 0.2)     # mean=1.0, std=0.2
aicon.add_continuous_factor("average_order_value", 50.0, 10.0)  # mean=50.0, std=10.0

# Create an action space for budget allocation
aicon.create_action_space(
    action_type="budget_allocation",
    total_budget=1000.0,
    items=["ad_1", "ad_2", "ad_3"]
)

# Create a marketing ROI utility function
aicon.create_utility_function(
    utility_type="marketing_roi",
    revenue_per_sale=100.0  # Assuming $100 revenue per sale
)

# Print the current state
print("\nCurrent AIcon State:")
print(f"Factors: {aicon.factors}")
print(f"Action Space: {aicon.action_space}")
print(f"Utility Functions: {aicon.utility_functions}")

# Try to find the best action
try:
    best_action = aicon.find_best_action()
    print("\nBest Action Found:")
    print(best_action)
except Exception as e:
    print(f"\nError finding best action: {e}")